In [1]:
# Get miscellaneous system information
import os
import sys

# Returns the current conda environment if found, 'None' if not found
def get_conda_env():
    sp = sys.path[1].split('/')
    if 'envs' in sp:
        return sp[sp.index("envs") + 1]
    else:
        return None
# Get the current Anaconda Environment name
conda_env = get_conda_env()
print("Anaconda Environment: {}\n".format(conda_env))

# Get the Python version that is currently running
python_version = sys.version
print("Python Version: {}\n".format(python_version))

# Get the path to the project folder
project_path = os.getcwd()
print("project folder path: {}\n".format(project_path))

# Print out the number of items in the project folder
print("There are {0} items in the project folder:\n".format(len(os.listdir())))
for item in os.listdir():
     print(" - ", item)

Anaconda Environment: P5

Python Version: 3.5.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:11:22) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]

project folder path: /Users/ty/Udacity/T1/P5_vehicle_detection_and_tracking

There are 18 items in project folder:

 -  P5_vehicle_detection_and_tracking.ipynb
 -  test_images
 -  bbox-example-image.jpg
 -  output_images
 -  LICENSE
 -  project_video.mp4
 -  launch_P5.sh
 -  README.md
 -  .gitignore
 -  examples
 -  writeup_template.md
 -  numpy_basics.py
 -  .ipynb_checkpoints
 -  bbox-example-image-out.jpg
 -  .git
 -  lecture_notes.ipynb
 -  draw_boxes.py
 -  test_video.mp4
